In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import torch
import open_clip
import numpy as np
import pandas as pd
from PIL import Image
from torchvision import transforms
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [3]:
file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePUUM/1. Completed_Data/CanonBeetles.csv"
canon_df = pd.read_csv(file_path)

In [4]:
canon_df["ImageFilePath"] = canon_df["cropped_image_path"].apply(lambda x: f"/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePUUM/CANON/{x}")

In [5]:
cols = ['ImageFilePath', 'ScientificName']
df = canon_df[cols]
df.head(2)

,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax konanus
1,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax konanus


In [6]:
model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms("hf-hub:imageomics/bioclip")
tokenizer = open_clip.get_tokenizer("hf-hub:imageomics/bioclip")

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

for param in model.parameters() :
    param.requires_grad = False
    

In [7]:
def extract_image_features(image_path):
    
    image = Image.open(image_path).convert("RGB")

    image_tensor = preprocess_val(image).unsqueeze(0).to(device)

    with torch.no_grad():
        image_features = model.encode_image(image_tensor)
        image_features /= image_features.norm(dim=-1, keepdim=True)

    return image_features.cpu().numpy()


In [8]:
X = np.vstack([extract_image_features(img) for img in df["ImageFilePath"]])

print(f"Extracted feature shape: {X.shape}")

# Encode labels
le = LabelEncoder()

y = le.fit_transform(df["ScientificName"])

df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()


Extracted feature shape: (1622, 512)


In [9]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef


seed = 99


models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "SVMLinear": SVC(kernel="linear"),
    "SVMPolynomial": SVC(kernel="poly", degree=4),
    "SVMRadialBasis": SVC(kernel="rbf", degree=4),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=seed),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}


for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    predictions[name] = preds

    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)

    metrics[name] = {"Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}

    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T


NaiveBayes                | Acc: 86.15% | Prec: 89.68% | Rec: 86.15% | F1: 86.85% | Bal Acc: 66.12% | MCC: 0.8086
LogisticRegression        | Acc: 83.69% | Prec: 80.61% | Rec: 83.69% | F1: 79.67% | Bal Acc: 31.12% | MCC: 0.7453
SVMLinear                 | Acc: 88.92% | Prec: 83.64% | Rec: 88.92% | F1: 85.68% | Bal Acc: 48.28% | MCC: 0.8307
SVMPolynomial             | Acc: 92.00% | Prec: 89.75% | Rec: 92.00% | F1: 89.57% | Bal Acc: 53.22% | MCC: 0.8781
SVMRadialBasis            | Acc: 90.15% | Prec: 84.84% | Rec: 90.15% | F1: 86.97% | Bal Acc: 50.24% | MCC: 0.8506
NearestNeighbor           | Acc: 89.23% | Prec: 89.31% | Rec: 89.23% | F1: 87.84% | Bal Acc: 56.85% | MCC: 0.8352
RandomForest              | Acc: 83.69% | Prec: 82.29% | Rec: 83.69% | F1: 79.25% | Bal Acc: 34.32% | MCC: 0.7487
MLP_Baseline              | Acc: 92.92% | Prec: 92.08% | Rec: 92.92% | F1: 92.33% | Bal Acc: 61.52% | MCC: 0.8917


In [10]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_SVMLinear,Pred_SVMPolynomial,Pred_SVMRadialBasis,Pred_NearestNeighbor,Pred_RandomForest,Pred_MLP_Baseline
135,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax discedens,Mecyclothorax discedens,Mecyclothorax discedens,Mecyclothorax discedens,Mecyclothorax discedens,Mecyclothorax discedens,Mecyclothorax discedens,Mecyclothorax discedens,Mecyclothorax discedens
844,/fs/ess/PAS2136/Hawaii-2025/beetles_intake/Bee...,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus,Mecyclothorax konanus


In [11]:
test_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Beetle-PUUM/5.BioCLIP-linear-probing-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/3. Benchmarking/Beetle-PUUM/5.BioCLIP-linear-probing-species-metrics.csv", index=False)